<h1 style="color:#4E2A84;"> IQM - Qiskit - IQM - Mid Circuit Measurement and Qubit Reset</h1>

This notebook provides an example of using performing mid circuit measurement and then using the measurement results to reset the qubits on IQM. The mid circuit measurement is combined with a conditional $X$ gate to flip the qubits to $\ket{0}$ state thus achieving same result as active reset.

This notebook leverages the `IQM` cloud service, specifically IQM Resonance, it uses the `Qiskit` development environment and targets a `IQM` Garnet superconducting quantum computer.

In [ ]:
# reset all notebook variables just in case
%reset -f

<h1 style="color:#4E2A84;">Install IQM Qiskit Client and Other Dependencies</h1>

The IQM quantum computers used in this example are accessed via the IQM client, which is part of the `iqm-client[qiskit]` package.

In addition you need `pylatexenc` package when using the `Python 3 \[Default\]` environment. 

All the dependencies can be installed using `pip install` commands below.

In [ ]:
%%capture
!pip install -U "iqm-client[qiskit]==23.8"
!pip install -U "pylatexenc"

<h1 style="color:#4E2A84;">Provide IQM API Token</h1>

In order to access IQM Garnet quantum computer through the IQM Resonance cloud service, you will need to create and provide an API token from IQM. If you have IQM Resonance account, you can access your account and token by going to the [IQM Resonance Dashboard](https://resonance.meetiqm.com) and then clicking View Account on the top-right of the web page.

<h1 style="color:#4E2A84;">Request IQM Token</h1> 

If you do not have IQM Resonance account, please request access from the instructor. After you have IQM Resonance account, then you can generate your API token and enter it when the below code is executed.

In [ ]:
# input your IQM token
import os
os.environ["IQM_TOKEN"] = input()

<h1 style="color:#4E2A84;">Connect to IQM Quantum Computer</h1> 

Below code uses IQMProvider to connect to a quantum computer through an IQM server. Your API access token is used implicitly when establishing the connection.

You may ignore errors about IQM Client versions that may be printed when connecting to the server.

In [ ]:
# import IQMProvider package
from iqm.qiskit_iqm import IQMProvider

# server address for IQM Garnet
server_url = 'https://cocos.resonance.meetiqm.com/garnet'

# connect to server and obtain backend object used later to execute jobs
print(f"Create CoCoS backend: {server_url}")
provider = IQMProvider(server_url)
backend = provider.get_backend()

<h1 style="color:#4E2A84;">Sample Circuit with Mid Circuit Measurement and Conditional X Gate</h1>

Below is a simple circuit written using Qiskit that contains a single $H$ gate and then performs mid circuit measurement followed by conditional $X$ gate and a final measurement.

Note the circuit uses a concept of `barrier'. A barrier is used in quantum circuits to prevent the compiler from reordering or optimizing across the barrier in the circuit. It's a logical fence that ensures operations before and after the barrier remain in their specified order when the circuit is transpiled. E.g. operations after the barrier are never moved to happen before the barrier.

In [ ]:
# import Qiskit packages
from qiskit import visualization, QuantumCircuit, ClassicalRegister, QuantumRegister, transpile
from qiskit.visualization import plot_histogram

# define qubits and classical registers
qubit_1 = QuantumRegister(1, "Q1")
qubit_2 = QuantumRegister(1, "Q2")
c1 = ClassicalRegister(1, "c1")
c2 = ClassicalRegister(1, "c2")
c3 = ClassicalRegister(1, "c3")

# create circuit
circ = QuantumCircuit(qubit_1, qubit_2, c1, c2, c3)

# simple circuit with one H age
circ.h(qubit_1)

# measure the qubit and conditionally apply X gate
circ.measure(qubit_1, c1)
circ.barrier()
circ.x(qubit_1).c_if(c1 ,1)

# again, measure the qubit and conditionally apply X gate
circ.measure(qubit_1, c2)
circ.barrier()
circ.x(qubit_1).c_if(c2 ,1)

# final measurement
circ.measure(qubit_1, c3)

In [ ]:
# dispaly circuit for visual inspection
display(circ.draw('mpl'))

<h1 style="color:#4E2A84;">Transpile the Circuit</h1> 

As a next step, you need to transpile (i.e. compile) the circuit for the target IQM Garnet quantum computer.

In [ ]:
# transpile the circuit
transpiled_circ = transpile(circ, backend=backend, optimization_level=0)

In [ ]:
# dispaly circuit for visual inspection
transpiled_circ.draw('mpl')

<h1 style="color:#4E2A84;">Subimt Circuit for Execution on Quantum Computer</h1> 

Now you can submit the circuit for execution. The circuit is executed by calling the `backend.run()` function. When the function is executed, the circuit will be queued by IQM Resonance for execution. Note that IQM has periods (hours) when the machine is not available, so you may need to wait a long time to get results, depending on what time of day you submit the circuit. You can check the $DEVICES$ tab on the right-hand side of the qBraid Lab to check when IQM will be available.

In [ ]:
# set number of shots
shots = 1000

# submit circuit for execution
job = backend.run(transpiled_circ, shots=shots)

# get job id for reference
job_id = job.job_id()
print('Job id:', job_id)

<h1 style="color:#4E2A84;">Wait for Circuit to Execute</h1> 

You can regularly (or irregularly) check the status of this job by executing the following code. Job status (and results) are retrieved using IQM Client object. A connection is made to the IQM Resonance server (with your API token again being used implicitly), after which you can query the job status. You can alternatively check the job status on the [IQM Resonance Dashboard](https://resonance.meetiqm.com).

You may ignore errors about IQM Client versions that may be printed when connecting to the server.

When the output is `Status.READY` the job has completed and you can access the results.

In [ ]:
# import IQM Client package
from iqm.iqm_client.iqm_client import IQMClient

# server address
server_url = 'https://cocos.resonance.meetiqm.com/garnet'

# create IQM Client object
iqm_client = IQMClient(server_url)

# get status of desired job
if job_id is None:
    job_id = ''
job_run_status = iqm_client.get_run_status(job_id)

# print status
print(job_run_status.status)

<h1 style="color:#4E2A84;">Analyze Results</h1>

In [ ]:
# get job results
job_results = iqm_client.wait_for_results(job_id)

# plot result
result_dict = job.result().get_counts()
visualization.plot_histogram(result_dict)

In [ ]:

# Copyright 2025 Jakub Szefer
# 
# Based on code from:
# https://www.iqmacademy.com/notebookViewer/?path=%2Fnotebooks%2Fiqm%2Fgeneral%2FControlledPRXQiskit.ipynb
#
# Changes:
# * add detailed comments for each step
# * use IQMClient to check job status and separately retrieve the results when job completes
# 
# Copyright 2025 IQM Quantum Computers (Stefan Pogorzalek, Stefan Seegerer)
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.